## Part 1 of the Assignment - Creating the dataframe

### Importing libraries and extracting  table

In [ ]:
import pandas as pd

# Webpage url        
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

# Extract tables
dfs = pd.read_html(url)

# print number of tables
print(len(dfs))

# Get first table which is the table of interest                                                                                                           
df = dfs[0]

### Extract Required columns into a df

In [ ]:
# Extract required columns                                                                                                           
df2 = df[['Postal Code','Borough','Neighbourhood']]


### Ignore cells with a borough that is Not assigned

In [ ]:
# get rid of rows with Borough value 'Not assigned'
df2 = df2[df2.Borough != 'Not assigned'].reset_index(drop=True)


### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough

In [ ]:
mask = df2['Neighbourhood'] == "Not assigned"
df2.loc[mask,'Neighbourhood'] = df2.loc[mask, 'Borough']

### print number of rows of the df

In [ ]:
print(df2.shape[0])


### Display dataframe

In [ ]:
df2.head(12)

## Part 2 of the assignement - obtaining latitudes and longitudes

### read csv file with longitude an latitude details

In [ ]:
df_lng_lat = pd.read_csv('Geospatial_Coordinates.csv')
df_lng_lat.head()

### Merge two dataframes with the common column latitude and longitude

In [ ]:
df_merged = df2.merge(df_lng_lat, on="Postal Code", how = 'left')
df_merged.head()

In [ ]:
print(df_merged.shape[0])

## Part 3 of the assignment - Explore and cluster the neighborhoods in Toronto.

### Extracting boroughs that contain the word Toronto 

In [ ]:
df_merged = df_merged[df_merged['Borough'].str.contains("Toronto")]
df_merged.head()

### Create a map of Toronto with neighborhoods superimposed on top.

In [ ]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

# !conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
# from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

In [ ]:

latitude = 43.651070
longitude = -79.347015

# create map of New York using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_merged['Latitude'], df_merged['Longitude'], df_merged['Borough'], df_merged['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

### Define Foursquare Credentials and Version

In [ ]:
CLIENT_ID = 'GURYN0HXLCV2RLRBQZSKURSEVN5ZVZTB14HYM5DKEON3KGSW' # your Foursquare ID
CLIENT_SECRET = 'W54MVLZU1PPZFODSDSKH3LDDMIZEIRZMCNXXDBNQ5OQPEFB3' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

### Explore Neighborhoods in Toronto

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
Toronto_venues = getNearbyVenues(names=df_merged['Neighbourhood'],
                                   latitudes=df_merged['Latitude'],
                                   longitudes=df_merged['Longitude']
                                  )

### Cluster the neighborhoods

In [ ]:
# set number of clusters
kclusters = 5

# toronto_grouped_clustering = df_merged.drop('Neighbourhood', 1)

toronto_grouped_clustering = df_merged.drop(['Neighbourhood', 'Borough', 'Postal Code'], axis=1)


# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 
print(len(kmeans.labels_))
print(toronto_grouped_clustering.shape[0])

In [ ]:
# add clustering labels
df_merged.insert(0, 'Cluster Labels', kmeans.labels_)
df_merged.head()

### Display clusters on map

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_merged['Latitude'], df_merged['Longitude'], df_merged['Neighbourhood'], df_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters